## This script will develop and test a multiple regression model for GHD-Core using the following predictors:
#### CRU Instrumental Temperature, Precipitation, PDSI

#### Season: May-June-July (MJJ)
#### Years: 1901-1980

* Reset the python session to clear out all variables, etc.

In [1]:
%reset -f

* Import all the various python modules that I will need in order to run the analyses and generate the figures.

In [2]:
# Import Modules and define functions
import calendar
import os
import numpy as np
import netCDF4
import matplotlib
import copy
from matplotlib import pyplot as plt
import scipy
import seaborn as sns
import pandas as pd
import scipy.stats as stats
from IPython.display import display
from mpl_toolkits.basemap import Basemap, cm
#sns.set(palette="Set5")

# STATSMODELS Package: needed for multiple regression
import statsmodels.api as sm

# Embeds plots inside the notebook (use in iPython Notebook)
%matplotlib inline

# For plotting a rectangle on the maps
def plot_rectangle(bmap, lonmin,lonmax,latmin,latmax):
    xs = [lonmin,lonmax,lonmax,lonmin,lonmin]
    ys = [latmin,latmin,latmax,latmax,latmin]
    bmap.plot(xs, ys,latlon = True, color='k', linestyle='--', linewidth=3)
    

* Set some various user variables.
* Load the GHD-Core data and set season and years to analyze.

In [3]:
# Setup the Analysis

# User Set Variables (knobs)
# Months to average (climate data)
mons_ave = [5,6,7]; mons_ave_txt = 'MJJ';
yr1 = np.array([1901]); yr2 = np.array([1980])  

# Rectangle Boundaries (Also the region over which I will spatially average the CRU data)
lonmin=-2; lonmax=8; latmin=43; latmax=51;

# Cru Lat/Lon Range (boundaries for the Map)
lat1_cru = np.array([27]);   lat2_cru = np.array([71])   
lon1_cru = np.array([-12]);  lon2_cru = np.array([45])  

# GHD Data: All Sites I want to Analyze
ghd_all_names=[ 'GHDcore',\
            ]

# Load formatted GHD anomaly data into a dataframe
infile= '../data/ghd_anom_doy_v02.csv'   # Name of the original data file
df=pd.read_csv(infile)

# Pull out the year vector
yr = np.int64(df.Year)

# Load lat/lon data for individual Sites
infile = '../data/site_locs.csv'   # Name of the data file
df_sitelocs=pd.read_csv(infile)
df_sitelocs.index=df_sitelocs.Location


* In this cell, I will load and seasonally average/sum the CRU temperature and precipitation data. As part of this, I am creating month and year vectors so that I can pull the specific data I want. This is a more or less global dataset, so I also trim the latitude and longitude ranges for just Europe, where the GHD data is located.

In [4]:
# LOAD CRU TEMP PRECIP Data. These are from version 3.21 of the CRU Climate Grids.

# Create vectors for years and months
mon_cru = np.arange(1,13); mon_all = np.transpose(np.tile(mon_cru,(1,112)))
yr_cru  = np.arange(1901,2013); yr_all = np.transpose(np.tile(yr_cru,(12,1))); yr_all = np.reshape(yr_all,(1344,1))

# Open and Load Data from a NetCDF File
ncfile_tmp = netCDF4.Dataset('/Users/bcook/Documents/GEODATA/cru321/cru_ts3.21.1901.2012.tmp.dat.nc')
ncfile_pre = netCDF4.Dataset('/Users/bcook/Documents/GEODATA/cru321/cru_ts3.21.1901.2012.pre.dat.nc')

# Load in the dimension variables
lon = ncfile_tmp.variables['lon'][:]
lat = ncfile_tmp.variables['lat'][:]

# Just load region I want to look at
# Yr/Lat/Lon Location for Grid Cell I want
i_lat = np.where((lat>=lat1_cru) & (lat<=lat2_cru)); i_lat=i_lat[0]
i_lon = np.where((lon>=lon1_cru) & (lon<=lon2_cru)); i_lon=i_lon[0]

# Initialize Matrices for Climate Variables
tmp_month = np.zeros((np.size(mons_ave),np.size(yr_cru),np.size(i_lat),np.size(i_lon)))
pre_month = np.zeros((np.size(mons_ave),np.size(yr_cru),np.size(i_lat),np.size(i_lon)))

# Load this region for the months requested
for i in enumerate(mons_ave):
    # Location for all instances of the current month    
    loc_mon = np.where(mon_all==i[1]); loc_mon=loc_mon[0];

    # These files are organized: time, lat, lon
    tmp=ncfile_tmp.variables['tmp'][loc_mon,i_lat,i_lon];
    pre=ncfile_pre.variables['pre'][loc_mon,i_lat,i_lon];

    # Store the Monthly Data
    tmp_month[i[0],:,:,:] = tmp
    pre_month[i[0],:,:,:] = pre    
    
    print(i)

# Now calculate seasonal average/sum
tmp_seas_ave = np.nanmean(tmp_month,axis=0)
pre_seas_sum = np.nansum(pre_month,axis=0)


(0, 5)
(1, 6)
(2, 7)


* Just like the previous cell, but here I load the PDSI data, calculated from the CRU climate grids by Gerard van der Schier. The files are in a bit of a different format, so this had to be its' own code.

In [5]:
# LOAD CRU scPDSI Data. These are based on 3.21 of the CRU Climate Grids, calculated by Gerard van der Schrier.

# Create vectors for years and months
mon_cru = np.arange(1,13);      mon_all = np.transpose(np.tile(mon_cru,(1,112)))
yr_cru  = np.arange(1901,2013); yr_all  = np.transpose(np.tile(yr_cru,(12,1))); yr_all = np.reshape(yr_all,(1344,1))

# PDSI data
root_dir = '/Users/bcook/Documents/GEODATA/cru321/'

# CRU PDSI are split up among different files, so I will have to load each one
# individually
files_crupdsi = ['pdsi.3.21.penman.snow.1901-1910.nc', \
    'pdsi.3.21.penman.snow.1911-1920.nc',\
    'pdsi.3.21.penman.snow.1911-1920.nc',\
    'pdsi.3.21.penman.snow.1931-1940.nc',\
    'pdsi.3.21.penman.snow.1941-1950.nc',\
    'pdsi.3.21.penman.snow.1951-1960.nc',\
    'pdsi.3.21.penman.snow.1961-1970.nc',\
    'pdsi.3.21.penman.snow.1971-1980.nc',\
    'pdsi.3.21.penman.snow.1981-1990.nc',\
    'pdsi.3.21.penman.snow.1991-2000.nc',\
    'pdsi.3.21.penman.snow.2001-2010.nc',\
    'pdsi.3.21.penman.snow.2011-2012.nc'\
    ]

# Load in the dimension variables
fname = root_dir+files_crupdsi[0]
ncfile_pdsi = netCDF4.Dataset(fname)
lon = ncfile_pdsi.variables['lon'][:]
lat = ncfile_pdsi.variables['lat'][:]
ncfile_pdsi.close

# Trim lons/lats
i_lat = np.where((lat>=lat1_cru) & (lat<=lat2_cru)); i_lat=i_lat[0]
i_lon = np.where((lon>=lon1_cru) & (lon<=lon2_cru)); i_lon=i_lon[0]
lon_map = lon[i_lon]; lat_map = lat[i_lat];

# Load Each file separately
for ifile in np.arange(0,np.size(files_crupdsi)):
    
    # Current File Name/Open netcdf object
    fname = root_dir+files_crupdsi[ifile]; print(fname)
    ncfile_pdsi = netCDF4.Dataset(fname)
    pdsi = np.float64(ncfile_pdsi.variables['pdsi'][:,i_lat,i_lon]);

    # Concatenate into a complete array for all files
    if ifile==0:
        pdsi_all=pdsi;
    else:
        pdsi_all=np.concatenate((pdsi_all,pdsi),axis=0)

    # Close netcdf file
    ncfile_pdsi.close

#sns.plt.plot(pdsi_all[:,44,60])

# Now pull out and calculate seasonal averages
# Initialize Matrices for Climate Variables
pdsi_month = np.zeros((np.size(mons_ave),np.size(yr_cru),np.size(i_lat),np.size(i_lon)))

# Load this region for the months requested
for i in enumerate(mons_ave):
    print(i)
    # Location for all instances of the current month    
    loc_mon = np.where(mon_all==i[1]); loc_mon=loc_mon[0];

    # Store in a new matrix
    pdsi_month[i[0],:,:,:] = pdsi_all[loc_mon,:,:]

# Seasonal average PDSI
pdsi_seas = np.nanmean(pdsi_month,axis=0)

# Delete Variables I don't need anymore
del(pdsi_month)
del(pdsi_all)
del(pdsi)


/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1901-1910.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1911-1920.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1911-1920.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1931-1940.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1941-1950.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1951-1960.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1961-1970.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1971-1980.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1981-1990.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.1991-2000.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.2001-2010.nc
/Users/bcook/Documents/GEODATA/cru321/pdsi.3.21.penman.snow.2011-2012.nc
(0, 5)
(1, 6)
(2, 7)


* All the data should now be loaded. This cell just does a quick check to make sure the format of the climate data looks okay.

In [6]:
# All the data should now be loaded by this point
# Location (indices) of Years to Correlate
# 
print('All the climate data is now loaded, and averaged to create the seasonal average/sums.')
print('They should be arrayed as: yr x lat x lon ')
print('')
print('Yrs: '+np.str(np.min(yr_cru))+'-'+np.str(np.max(yr_cru)))
print('Lat: '+np.str(np.min(lat_map))+' to '+np.str(np.max(lat_map)))
print('Lon: '+np.str(np.min(lon_map))+' to '+np.str(np.max(lon_map)))
print('')
print('Temp Array Size: '+np.str(tmp_seas_ave.shape))
print('Prec Array Size: '+np.str(pre_seas_sum.shape))
print('PDSI Array Size: '+np.str(pdsi_seas.shape))


All the climate data is now loaded, and averaged to create the seasonal average/sums.
They should be arrayed as: yr x lat x lon 

Yrs: 1901-2012
Lat: 27.25 to 70.75
Lon: -11.75 to 44.75

Temp Array Size: (112, 88, 114)
Prec Array Size: (112, 88, 114)
PDSI Array Size: (112, 88, 114)


* For the regression analyses, I need time series. This next cell will calculate regional average time series (cosine area weighted by latitude) from the CRU data. For GHD-Core, I pick a big area that encompasses most of France.

In [7]:
# Calculate Regional Averages for CRU Data around each of the GHD Locations. For the Core and Composite Indices,
# use a large region covering France.

# Arrays to store regional average PDSI/Temp/Precip for each site
pdsi_coswtmean = np.zeros((np.size(yr_cru),np.size(ghd_all_names)))
tmp_coswtmean  = np.zeros((np.size(yr_cru),np.size(ghd_all_names)))
pre_coswtmean  = np.zeros((np.size(yr_cru),np.size(ghd_all_names)))

# Loop through each site individually.
for ifile in enumerate(ghd_all_names):
    # Counter
    print(ifile)
    
    # Pull GHD Site Coordinates (if specific site)
    #        Dummy Coorindates for Core and Composite Index
    if ifile[1] in ['GHDcore','GHDmean']:
        
        # Latitude/Longitude range for averaging
        print("lat range = ("+np.str(latmin)+" to "+np.str(latmax)+")")
        print("lon range = ("+np.str(lonmin)+" to "+np.str(lonmax)+")")
       
        i_lat_reg = np.where((lat_map>=latmin) & (lat_map<=latmax)); i_lat_reg=i_lat_reg[0]
        i_lon_reg = np.where((lon_map>=lonmin) & (lon_map<=lonmax)); i_lon_reg=i_lon_reg[0]

        # Latitude and Longitude Indices for this region
        lon_reg = lon_map[i_lon_reg]
        lat_reg = lat_map[i_lat_reg]

        # Create Latitude Weights
        lat_wts = scipy.cos(scipy.deg2rad(lat_reg));
        lat_wts_grid,lon_junk = np.meshgrid(lat_wts,lon_reg)
        lat_wts_grid=np.swapaxes(lat_wts_grid,1,0)
        
    # Load Each Year and Spatially Average
    for i_yr in enumerate(yr_cru):
        #print(i_yr)
        # Pull out Current Month Temp/Precip
        pdsi_curr = pdsi_seas[i_yr[0],i_lat_reg,:][:,i_lon_reg]
        temp_curr = tmp_seas_ave[i_yr[0],i_lat_reg,:][:,i_lon_reg]
        prec_curr = pre_seas_sum[i_yr[0],i_lat_reg,:][:,i_lon_reg]

        # Mask ocean cells
        prec_curr[prec_curr>=100000]=np.nan
        temp_curr[temp_curr>=100000]=np.nan
        pdsi_curr[pdsi_curr>=100000]=np.nan
        
        # Cosine Weighted Average
        pdsi_coswtmean[i_yr[0],ifile[0]] = np.ma.average(np.ma.masked_invalid(pdsi_curr),weights=lat_wts_grid)
        tmp_coswtmean[i_yr[0],ifile[0]]  = np.ma.average(np.ma.masked_invalid(temp_curr),weights=lat_wts_grid)
        pre_coswtmean[i_yr[0],ifile[0]]  = np.ma.average(np.ma.masked_invalid(prec_curr),weights=lat_wts_grid)


(0, 'GHDcore')
lat range = (43 to 51)
lon range = (-2 to 8)


#### Regression Analysis: GHD-Core
* I am going to conduct single variable regressions, and multivariate regressions using temperature and different combinations of prec and temp. The first thing I will do, however, is set up a dataframe to store the results.

In [8]:
# Create Dataframe to store regression results

# Column Headers
df_cols =   [   'R2',\
                'AIC',\
                'BIC',\
                'x1 (p)',\
                'x2 (p)',\
                'x3 (p)',\
            ]

# Row labels
df_rows =   [   'Temp',\
                'Prec',\
                'PDSI',\
                'Temp+Prec',\
                'Temp+PDSI',\
                'Temp+Prec+PDSI',\
                'Temp+Prec+(Tmp x Pre)',\
                'Temp+PDSI+(Tmp x PDSI)',\
           ]

# Create the dataframe-at this point, it should just be filled with NaN placeholders.
df_regress = pd.DataFrame(index=df_rows,columns=df_cols)
df_regress

,R2,AIC,BIC,x1 (p),x2 (p),x3 (p)
Temp,NaN,NaN,NaN,NaN,NaN,NaN
Prec,NaN,NaN,NaN,NaN,NaN,NaN
PDSI,NaN,NaN,NaN,NaN,NaN,NaN
Temp+Prec,NaN,NaN,NaN,NaN,NaN,NaN
Temp+PDSI,NaN,NaN,NaN,NaN,NaN,NaN
Temp+Prec+PDSI,NaN,NaN,NaN,NaN,NaN,NaN
Temp+Prec+(Tmp x Pre),NaN,NaN,NaN,NaN,NaN,NaN
Temp+PDSI+(Tmp x PDSI),NaN,NaN,NaN,NaN,NaN,NaN


#### Pull out climate and GHD time series for just the years that I want.

In [9]:
# Pull out the relevant time series for the years I want (climate and GHD)
# Non detrended data
# Year Locations of the GHD data and climate data.
loc_yrs_ghd = np.where((yr>=yr1)     & (yr<=yr2))[0];
loc_yrs_cru = np.where((yr_cru>=yr1) & (yr_cru<=yr2))[0];

# Pull out GHD
ghd=df.GHDcore; ghd=ghd[loc_yrs_ghd];
 
# Pull out Climate Data
tmp_series  = tmp_coswtmean[loc_yrs_cru]
pre_series  = pre_coswtmean[loc_yrs_cru]
pdsi_series = pdsi_coswtmean[loc_yrs_cru]

### TEMPERATURE REGRESSION

In [10]:
## REGRESSION TEMPERATURE PREDICTOR ONLY---------------------------------------------
# Remember to manually add a constant term
predictor_array      = np.zeros([np.size(tmp_series),1])*np.nan   
predictor_array[:,0] = tmp_series[:,0]
predictor_array      = sm.tools.add_constant(predictor_array)

model = sm.OLS(ghd,predictor_array)
results = model.fit()

# Pull out relevant information from the regression
R2     = np.round(results.rsquared,decimals=3)
AIC    = np.round(results.aic,decimals=2)
BIC    = np.round(results.bic,decimals=2)
x1     = np.round(results.params.x1,decimals=2)
x1_p   = np.round(results.pvalues.x1,decimals=5)
x1_str = np.str(x1)+' ('+np.str(x1_p)+')'

# Store data in this dataframe
df_regress.values[0,0] = R2
df_regress.values[0,1] = AIC
df_regress.values[0,2] = BIC
df_regress.values[0,3] = x1_str

### PRECIPITATION REGRESSION

In [11]:
## REGRESSION PRECIPITATION PREDICTOR ONLY---------------------------------------------
# Remember to manually add a constant term
predictor_array      = np.zeros([np.size(pre_series),1])*np.nan   
predictor_array[:,0] = pre_series[:,0]
predictor_array      = sm.tools.add_constant(predictor_array)

model = sm.OLS(ghd,predictor_array)
results = model.fit()

# Pull out relevant information from the regression
R2     = np.round(results.rsquared,decimals=3)
AIC    = np.round(results.aic,decimals=2)
BIC    = np.round(results.bic,decimals=2)
x1     = np.round(results.params.x1,decimals=2)
x1_p   = np.round(results.pvalues.x1,decimals=5)
x1_str = np.str(x1)+' ('+np.str(x1_p)+')'

# Store data in this dataframe
df_regress.values[1,0] = R2
df_regress.values[1,1] = AIC
df_regress.values[1,2] = BIC
df_regress.values[1,3] = x1_str

### PDSI REGRESSION

In [12]:
## REGRESSION PDSI PREDICTOR ONLY---------------------------------------------
# Remember to manually add a constant term
predictor_array      = np.zeros([np.size(pdsi_series),1])*np.nan   
predictor_array[:,0] = pdsi_series[:,0]
predictor_array      = sm.tools.add_constant(predictor_array)

model = sm.OLS(ghd,predictor_array)
results = model.fit()

# Pull out relevant information from the regression
R2     = np.round(results.rsquared,decimals=3)
AIC    = np.round(results.aic,decimals=2)
BIC    = np.round(results.bic,decimals=2)
x1     = np.round(results.params.x1,decimals=2)
x1_p   = np.round(results.pvalues.x1,decimals=5)
x1_str = np.str(x1)+' ('+np.str(x1_p)+')'

# Store data in this dataframe
df_regress.values[2,0] = R2
df_regress.values[2,1] = AIC
df_regress.values[2,2] = BIC
df_regress.values[2,3] = x1_str

### TEMPERATURE+PRECIPITATION REGRESSION

In [13]:
## REGRESSION TEMP+PREC ONLY---------------------------------------------
# I need to manually add a constant term
predictor_array      = np.zeros([np.size(tmp_series),2])*np.nan   
predictor_array[:,0] = tmp_series[:,0]
predictor_array[:,1] = pre_series[:,0]
predictor_array      = sm.tools.add_constant(predictor_array)

model = sm.OLS(ghd,predictor_array)
results = model.fit()

# Pull out relevant information from the regression
R2     = np.round(results.rsquared,decimals=3)
AIC    = np.round(results.aic,decimals=2)
BIC    = np.round(results.bic,decimals=2)

x1     = np.round(results.params.x1,decimals=2)
x1_p   = np.round(results.pvalues.x1,decimals=5)
x1_str = np.str(x1)+' ('+np.str(x1_p)+')'

x2     = np.round(results.params.x2,decimals=2)
x2_p   = np.round(results.pvalues.x2,decimals=5)
x2_str = np.str(x2)+' ('+np.str(x2_p)+')'

# Store data in this dataframe
df_regress.values[3,0] = R2
df_regress.values[3,1] = AIC
df_regress.values[3,2] = BIC
df_regress.values[3,3] = x1_str
df_regress.values[3,4] = x2_str

### TEMPERATURE+PDSI REGRESSION

In [14]:
## REGRESSION TEMP+PDSI ONLY---------------------------------------------
# I need to manually add a constant term
predictor_array      = np.zeros([np.size(tmp_series),2])*np.nan   
predictor_array[:,0] = tmp_series[:,0]
predictor_array[:,1] = pdsi_series[:,0]
predictor_array      = sm.tools.add_constant(predictor_array)

model = sm.OLS(ghd,predictor_array)
results = model.fit()

# Pull out relevant information from the regression
R2     = np.round(results.rsquared,decimals=3)
AIC    = np.round(results.aic,decimals=2)
BIC    = np.round(results.bic,decimals=2)

x1     = np.round(results.params.x1,decimals=2)
x1_p   = np.round(results.pvalues.x1,decimals=5)
x1_str = np.str(x1)+' ('+np.str(x1_p)+')'

x2     = np.round(results.params.x2,decimals=2)
x2_p   = np.round(results.pvalues.x2,decimals=5)
x2_str = np.str(x2)+' ('+np.str(x2_p)+')'

# Store data in this dataframe
df_regress.values[4,0] = R2
df_regress.values[4,1] = AIC
df_regress.values[4,2] = BIC
df_regress.values[4,3] = x1_str
df_regress.values[4,4] = x2_str

### TEMPERATURE+PRECIPITATION+PDSI REGRESSION

In [15]:
## REGRESSION TEMP+PREC+PDSI ONLY---------------------------------------------
# I need to manually add a constant term
predictor_array      = np.zeros([np.size(tmp_series),3])*np.nan   
predictor_array[:,0] = tmp_series[:,0]
predictor_array[:,1] = pre_series[:,0]
predictor_array[:,2] = pdsi_series[:,0]
predictor_array      = sm.tools.add_constant(predictor_array)

model   = sm.OLS(ghd,predictor_array)
results = model.fit()

# Pull out relevant information from the regression
R2     = np.round(results.rsquared,decimals=3)
AIC    = np.round(results.aic,decimals=2)
BIC    = np.round(results.bic,decimals=2)

x1     = np.round(results.params.x1,decimals=2)
x1_p   = np.round(results.pvalues.x1,decimals=5)
x1_str = np.str(x1)+' ('+np.str(x1_p)+')'

x2     = np.round(results.params.x2,decimals=2)
x2_p   = np.round(results.pvalues.x2,decimals=5)
x2_str = np.str(x2)+' ('+np.str(x2_p)+')'

x3     = np.round(results.params.x3,decimals=2)
x3_p   = np.round(results.pvalues.x3,decimals=5)
x3_str = np.str(x3)+' ('+np.str(x3_p)+')'

# Store data in this dataframe
df_regress.values[5,0] = R2
df_regress.values[5,1] = AIC
df_regress.values[5,2] = BIC
df_regress.values[5,3] = x1_str
df_regress.values[5,4] = x2_str
df_regress.values[5,5] = x3_str


### TEMPERATURE+PRECIPITATION+(Temp/Prec Interaction) REGRESSION

In [16]:
## REGRESSION TEMP+PREC+PDSI ONLY---------------------------------------------
# I need to manually add a constant term
predictor_array      = np.zeros([np.size(tmp_series),3])*np.nan   
predictor_array[:,0] = tmp_series[:,0]
predictor_array[:,1] = pre_series[:,0]
predictor_array[:,2] = tmp_series[:,0]*pre_series[:,0]
predictor_array      = sm.tools.add_constant(predictor_array)

model   = sm.OLS(ghd,predictor_array)
results = model.fit()

# Pull out relevant information from the regression
R2     = np.round(results.rsquared,decimals=3)
AIC    = np.round(results.aic,decimals=2)
BIC    = np.round(results.bic,decimals=2)

x1     = np.round(results.params.x1,decimals=2)
x1_p   = np.round(results.pvalues.x1,decimals=5)
x1_str = np.str(x1)+' ('+np.str(x1_p)+')'

x2     = np.round(results.params.x2,decimals=2)
x2_p   = np.round(results.pvalues.x2,decimals=5)
x2_str = np.str(x2)+' ('+np.str(x2_p)+')'

x3     = np.round(results.params.x3,decimals=2)
x3_p   = np.round(results.pvalues.x3,decimals=5)
x3_str = np.str(x3)+' ('+np.str(x3_p)+')'

# Store data in this dataframe
df_regress.values[6,0] = R2
df_regress.values[6,1] = AIC
df_regress.values[6,2] = BIC
df_regress.values[6,3] = x1_str
df_regress.values[6,4] = x2_str
df_regress.values[6,5] = x3_str


### TEMPERATURE+PDSI+(Temp/PDSI Interaction) REGRESSION

In [17]:
## REGRESSION TEMP+PREC+PDSI ONLY---------------------------------------------
# I need to manually add a constant term
predictor_array      = np.zeros([np.size(tmp_series),3])*np.nan   
predictor_array[:,0] = tmp_series[:,0]
predictor_array[:,1] = pdsi_series[:,0]
predictor_array[:,2] = tmp_series[:,0]*pdsi_series[:,0]
predictor_array      = sm.tools.add_constant(predictor_array)

model   = sm.OLS(ghd,predictor_array)
results = model.fit()

# Pull out relevant information from the regression
R2     = np.round(results.rsquared,decimals=3)
AIC    = np.round(results.aic,decimals=2)
BIC    = np.round(results.bic,decimals=2)

x1     = np.round(results.params.x1,decimals=2)
x1_p   = np.round(results.pvalues.x1,decimals=5)
x1_str = np.str(x1)+' ('+np.str(x1_p)+')'

x2     = np.round(results.params.x2,decimals=2)
x2_p   = np.round(results.pvalues.x2,decimals=5)
x2_str = np.str(x2)+' ('+np.str(x2_p)+')'

x3     = np.round(results.params.x3,decimals=2)
x3_p   = np.round(results.pvalues.x3,decimals=5)
x3_str = np.str(x3)+' ('+np.str(x3_p)+')'

# Store data in this dataframe
df_regress.values[7,0] = R2
df_regress.values[7,1] = AIC
df_regress.values[7,2] = BIC
df_regress.values[7,3] = x1_str
df_regress.values[7,4] = x2_str
df_regress.values[7,5] = x3_str


In [18]:
print("                  ")
print("                  ")
print("REGRESSION RESULTS")
df_regress

                  
                  
REGRESSION RESULTS


,R2,AIC,BIC,x1 (p),x2 (p),x3 (p)
Temp,0.704,421.72,426.48,-6.04 (0.0),NaN,NaN
Prec,0.241,496.99,501.75,0.07 (0.0),NaN,NaN
PDSI,0.133,507.7,512.46,1.68 (0.00089),NaN,NaN
Temp+Prec,0.713,421.27,428.41,-5.66 (0.0),0.02 (0.12563),NaN
Temp+PDSI,0.706,423.08,430.23,-5.9 (0.0),0.24 (0.4359),NaN
Temp+Prec+PDSI,0.713,423.26,432.79,-5.67 (0.0),0.02 (0.18964),-0.02 (0.94624)
Temp+Prec+(Tmp x Pre),0.717,422.2,431.73,-3.9 (0.03591),0.16 (0.27037),-0.01 (0.31657)
Temp+PDSI+(Tmp x PDSI),0.708,424.53,434.05,-6.14 (0.0),4.06 (0.44165),-0.24 (0.46842)


In [19]:
predictor_array      = np.zeros([np.size(tmp_series),3])*np.nan   
predictor_array[:,0] = tmp_series[:,0]
predictor_array[:,1] = pre_series[:,0]
predictor_array[:,2] = pdsi_series[:,0]

In [20]:
predictor_array

array([[  1.61641030e+01,   1.83988983e+02,  -6.95196608e-01],
       [  1.41644164e+01,   1.93765062e+02,  -1.00649784e+00],
       [  1.47939009e+01,   2.43210655e+02,  -1.08864630e+00],
       [  1.65363880e+01,   1.82431306e+02,  -1.26110351e+00],
       [  1.58818375e+01,   1.92479404e+02,  -1.95642044e+00],
       [  1.53848876e+01,   1.34320995e+02,  -1.80184117e+00],
       [  1.44780076e+01,   2.08453153e+02,  -2.00562414e+00],
       [  1.61110024e+01,   2.29382903e+02,  -1.38138184e+00],
       [  1.40949652e+01,   2.12171158e+02,  -8.81438429e-01],
       [  1.44956591e+01,   2.85954419e+02,   1.57503569e+00],
       [  1.65546708e+01,   1.88348924e+02,  -9.56207400e-01],
       [  1.54400258e+01,   2.17781601e+02,   7.48297873e-02],
       [  1.47877470e+01,   2.15855585e+02,   1.82205870e-01],
       [  1.45174120e+01,   2.70715107e+02,   8.82911201e-01],
       [  1.60582811e+01,   2.48426991e+02,   8.70065660e-01],
       [  1.47482395e+01,   2.07129043e+02,   8.1737106

In [22]:
ghd_array      = np.zeros([np.size(tmp_series),1])*np.nan   
ghd_array[:,0] = ghd


In [23]:
ghd_array

array([[ -9.61713869],
       [  3.02571845],
       [  4.01143274],
       [-10.40285298],
       [ -8.25999583],
       [ -8.01713869],
       [  2.58286131],
       [ -7.61713869],
       [ -1.55999583],
       [  1.02539042],
       [-13.96839949],
       [ -2.25589949],
       [ -4.24979512],
       [ -0.54231412],
       [-10.05170012],
       [  3.44829988],
       [-12.01777348],
       [ -3.80534655],
       [ -5.68089949],
       [ -6.29339949],
       [-11.76248941],
       [ -7.94820369],
       [  0.33751059],
       [ -6.74820369],
       [ -2.41963226],
       [ -0.89339949],
       [ -8.21963226],
       [ -4.51963226],
       [ -6.16248941],
       [ -2.76248941],
       [ -3.87313262],
       [  6.78160051],
       [ -5.62122369],
       [-14.66408083],
       [ -3.89265226],
       [ -1.52122369],
       [-11.5355094 ],
       [ -1.26408083],
       [  2.72163345],
       [ -3.50589949],
       [  1.60660051],
       [ -7.64339949],
       [-12.43089949],
       [ -5